In [164]:
import pandas as pd
from pandas import get_dummies

import numpy as np

from imblearn.over_sampling import SMOTE 

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [165]:
df = pd.read_csv("train.csv")
train_labels = df['Risk']
train_data = df.drop('Risk',axis = 'columns')
train_data.head()

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose
0,47,male,2,free,NaN,little,8335,36,car
1,38,male,2,own,little,NaN,804,12,radio/TV
2,28,male,2,own,little,little,5371,36,furniture/equipment
3,29,female,0,own,NaN,moderate,3990,36,domestic appliances
4,24,female,2,own,NaN,moderate,8487,48,car


In [166]:
test_df = pd.read_csv("test.csv")
test_labels = test_df['Risk']
test_data = test_df.drop('Risk', axis='columns')
test_data

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose
0,36,male,2,own,rich,moderate,1913,18,business
1,34,male,3,own,little,moderate,1860,12,car
2,48,male,2,own,little,little,1024,24,radio/TV
3,31,male,2,own,little,little,3104,18,business
4,23,male,1,own,quite rich,moderate,2520,27,radio/TV
...,...,...,...,...,...,...,...,...,...
195,53,male,2,free,little,little,7119,48,furniture/equipment
196,42,male,2,own,NaN,moderate,2427,18,business
197,25,male,2,own,little,NaN,1262,12,radio/TV
198,31,male,2,own,NaN,little,6350,30,furniture/equipment


In [167]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Age               800 non-null    int64 
 1   Sex               800 non-null    object
 2   Job               800 non-null    int64 
 3   Housing           800 non-null    object
 4   Saving accounts   656 non-null    object
 5   Checking account  484 non-null    object
 6   Credit amount     800 non-null    int64 
 7   Duration          800 non-null    int64 
 8   Purpose           800 non-null    object
dtypes: int64(4), object(5)
memory usage: 56.4+ KB


In [168]:
missing_value_cols = ['Saving accounts','Checking account']

In [169]:
train_data[missing_value_cols] = train_data[missing_value_cols].replace(np.nan,'NONE')
test_data[missing_value_cols] = test_data[missing_value_cols].replace(np.nan,'NONE')

In [170]:
train_data.dtypes

Age                  int64
Sex                 object
Job                  int64
Housing             object
Saving accounts     object
Checking account    object
Credit amount        int64
Duration             int64
Purpose             object
dtype: object

In [171]:
# for value in train_data['Credit amount']:
#     if value > 10000: value = 10000


# train_data['Credit amount'] = train_data['Credit amount'].map(lambda x: 10000 if x > 10000 else x)
# train_data['Credit amount'] = np.log(train_data['Credit amount']+1)

# print(train_data['Credit amount'].unique())

In [172]:
numeric_cols = ['Job','Credit amount','Duration']
categorical_cols = ['Sex','Housing','Saving accounts','Checking account','Purpose', 'Age']

In [173]:
def age_transform(value):
    if value <= 25: value = 1
    elif value <= 40 : value = 2
    elif value <50: value = 3
    elif value <80: value = 4
    else:
        value = 0
    return value

train_data['Age'] = train_data['Age'].map(age_transform)
test_data['Age'] = test_data['Age'].map(age_transform)



# def job_transform(value):
#     if value < 1 : value = 'No rent'
#     elif value < 2 : value = 'None skill - No rent'
#     elif value < 3: value = 'Skill - Rent'
#     elif value < 4 : value = 'High skill'
#     else: value = 'None'
#     return value

# train_data['Job'] = train_data['Job'].map(job_transform)
# test_data['Job'] = test_data['Job'].map(job_transform)


In [174]:
for col in categorical_cols:
    print(col,train_data[col].unique())

Sex ['male' 'female']
Housing ['free' 'own' 'rent']
Saving accounts ['NONE' 'little' 'moderate' 'rich' 'quite rich']
Checking account ['little' 'NONE' 'moderate' 'rich']
Purpose ['car' 'radio/TV' 'furniture/equipment' 'domestic appliances' 'education'
 'repairs' 'vacation/others' 'business']
Age [3 2 1 4]


In [175]:
train_data['Housing'] = train_data['Housing'].apply(lambda x: 0 if x == 'free' else (2 if x == 'own' else 1))
train_data['Saving accounts'] = train_data['Saving accounts'].apply(lambda x: 2 if x == 'nan' 
                                                                    else(0 if x == 'little' 
                                                                    else(1 if x == 'moderate'
                                                                    else(3 if x == 'quite rich' 
                                                                    else 4))))
                                                                    
train_data['Checking account'] = train_data['Checking account'].apply(lambda x: 2 if x == 'nan' 
                                                                    else(0 if x == 'little' 
                                                                    else(1 if x == 'moderate'
                                                                    else(3 if x == 'quite rich' 
                                                                    else 4))))

In [176]:
test_data['Housing'] = test_data['Housing'].apply(lambda x: 0 if x == 'free' else (2 if x == 'own' else 1))
test_data['Saving accounts'] = test_data['Saving accounts'].apply(lambda x: 0 if x == 'nan' 
                                                                    else(1 if x == 'little' 
                                                                    else(2 if x == 'moderate'
                                                                    else(3 if x == 'quite rich' 
                                                                    else 4))))
                                                                    
test_data['Checking account'] = test_data['Checking account'].apply(lambda x: 0 if x == 'nan' 
                                                                    else(1 if x == 'little' 
                                                                    else(2 if x == 'moderate'
                                                                    else(3 if x == 'quite rich' 
                                                                    else 4))))

In [177]:
numeric_cols = ['Age','Job','Credit amount','Duration','Housing','Saving accounts','Checking account']
categorical_cols = ['Sex','Purpose']

In [178]:
print(np.min(train_data['Credit amount']),np.max(train_data['Credit amount']))
print(np.min(test_data['Credit amount']),np.max(test_data['Credit amount']))

276 15945
250 18424


In [179]:
scaler = MinMaxScaler().fit(train_data[numeric_cols])
train_data[numeric_cols] = scaler.transform(train_data[numeric_cols])
train_data = get_dummies(train_data, categorical_cols)
train_data, train_labels = SMOTE().fit_resample(train_data, train_labels)
train_data

,Age,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Sex_female,Sex_male,Purpose_business,Purpose_car,Purpose_domestic appliances,Purpose_education,Purpose_furniture/equipment,Purpose_radio/TV,Purpose_repairs,Purpose_vacation/others
0,0.666667,0.666667,0.0,1.000000,0.000000,0.514328,0.571429,0,1,0,1,0,0,0,0,0,0
1,0.333333,0.666667,1.0,0.000000,1.000000,0.033697,0.142857,0,1,0,0,0,0,0,1,0,0
2,0.333333,0.666667,1.0,0.000000,0.000000,0.325164,0.571429,0,1,0,0,0,0,1,0,0,0
3,0.333333,0.000000,1.0,1.000000,0.250000,0.237029,0.571429,1,0,0,0,1,0,0,0,0,0
4,0.000000,0.666667,1.0,1.000000,0.250000,0.524028,0.785714,1,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115,0.333333,0.666667,1.0,0.220404,0.220404,0.065446,0.357143,0,1,0,0,0,0,0,1,0,0
1116,0.333333,0.666667,1.0,0.000000,0.338115,0.053334,0.250000,0,1,0,1,0,0,0,0,0,0
1117,0.629511,0.370489,1.0,0.777867,1.000000,0.156532,0.357315,0,1,1,0,0,0,0,0,0,0
1118,0.333333,0.666667,1.0,0.000000,0.183371,0.254362,0.571429,0,1,0,1,0,0,0,0,0,0


In [180]:
for col in train_data.columns:
    print(col,train_data[col].unique())

Age [0.66666667 0.33333333 0.         1.         0.46201469 0.71225397
 0.7631754  0.95024754 0.60368838 0.07894321 0.32388692 0.32480582
 0.73156409 0.34741818 0.00884422 0.81288155 0.32228293 0.75177302
 0.56122228 0.07231561 0.73965682 0.43175572 0.50695464 0.21155555
 0.3025061  0.22662302 0.42000966 0.42504377 0.28053263 0.97257007
 0.64723841 0.7422089  0.61402181 0.27016334 0.41423983 0.40434517
 0.47857968 0.3734362  0.48724828 0.28326718 0.32311975 0.12271335
 0.58128357 0.55107982 0.92012684 0.15070311 0.30360112 0.1045747
 0.43928591 0.20614399 0.38425039 0.26443032 0.66517069 0.31717387
 0.33522661 0.37206172 0.44394051 0.80428766 0.83264313 0.51969213
 0.27245213 0.60760502 0.02680394 0.47376153 0.15252792 0.22368981
 0.51636297 0.45558534 0.50753441 0.45280242 0.62000131 0.14422127
 0.09311054 0.7966352  0.13551299 0.81487357 0.72652454 0.24041439
 0.2217918  0.59521728 0.07509241 0.70003536 0.54099811 0.06308668
 0.38747616 0.45345205 0.06736891 0.56665793 0.88864278 0.1

In [181]:

test_data[numeric_cols] = scaler.transform(test_data[numeric_cols])
test_data = get_dummies(test_data, categorical_cols)
test_data


,Age,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Sex_female,Sex_male,Purpose_business,Purpose_car,Purpose_domestic appliances,Purpose_education,Purpose_furniture/equipment,Purpose_radio/TV,Purpose_repairs,Purpose_vacation/others
0,0.333333,0.666667,1.0,1.00,0.50,0.104474,0.250000,0,1,1,0,0,0,0,0,0,0
1,0.333333,1.000000,1.0,0.25,0.50,0.101091,0.142857,0,1,0,1,0,0,0,0,0,0
2,0.666667,0.666667,1.0,0.25,0.25,0.047738,0.357143,0,1,0,0,0,0,0,1,0,0
3,0.333333,0.666667,1.0,0.25,0.25,0.180484,0.250000,0,1,1,0,0,0,0,0,0,0
4,0.000000,0.333333,1.0,0.75,0.50,0.143213,0.410714,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1.000000,0.666667,0.0,0.25,0.25,0.436722,0.785714,0,1,0,0,0,0,1,0,0,0
196,0.666667,0.666667,1.0,1.00,0.50,0.137277,0.250000,0,1,1,0,0,0,0,0,0,0
197,0.000000,0.666667,1.0,0.25,1.00,0.062927,0.142857,0,1,0,0,0,0,0,1,0,0
198,0.333333,0.666667,1.0,1.00,0.25,0.387644,0.464286,0,1,0,0,0,0,1,0,0,0


In [182]:
feat_weight = np.full(17,1.0)
feat_weight[0:7] = [1,1,1,2,2,2,1.5]
feat_weight[7:9] = [0.5,0.5]
feat_weight[9:17] = [1/3]*8
feat_weight

array([1.        , 1.        , 1.        , 2.        , 2.        ,
       2.        , 1.5       , 0.5       , 0.5       , 0.33333333,
       0.33333333, 0.33333333, 0.33333333, 0.33333333, 0.33333333,
       0.33333333, 0.33333333])

In [183]:
i = 0
for col in train_data.columns:
    train_data[col] =  train_data[col]*feat_weight[i]
    test_data[col] =  test_data[col]*feat_weight[i]
    i+=1

In [184]:
train_data

,Age,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Sex_female,Sex_male,Purpose_business,Purpose_car,Purpose_domestic appliances,Purpose_education,Purpose_furniture/equipment,Purpose_radio/TV,Purpose_repairs,Purpose_vacation/others
0,0.666667,0.666667,0.0,2.000000,0.000000,1.028655,0.857143,0.0,0.5,0.000000,0.333333,0.000000,0.0,0.000000,0.000000,0.0,0.0
1,0.333333,0.666667,1.0,0.000000,2.000000,0.067394,0.214286,0.0,0.5,0.000000,0.000000,0.000000,0.0,0.000000,0.333333,0.0,0.0
2,0.333333,0.666667,1.0,0.000000,0.000000,0.650329,0.857143,0.0,0.5,0.000000,0.000000,0.000000,0.0,0.333333,0.000000,0.0,0.0
3,0.333333,0.000000,1.0,2.000000,0.500000,0.474057,0.857143,0.5,0.0,0.000000,0.000000,0.333333,0.0,0.000000,0.000000,0.0,0.0
4,0.000000,0.666667,1.0,2.000000,0.500000,1.048057,1.178571,0.5,0.0,0.000000,0.333333,0.000000,0.0,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115,0.333333,0.666667,1.0,0.440808,0.440808,0.130892,0.535714,0.0,0.5,0.000000,0.000000,0.000000,0.0,0.000000,0.333333,0.0,0.0
1116,0.333333,0.666667,1.0,0.000000,0.676231,0.106668,0.375000,0.0,0.5,0.000000,0.333333,0.000000,0.0,0.000000,0.000000,0.0,0.0
1117,0.629511,0.370489,1.0,1.555734,2.000000,0.313064,0.535972,0.0,0.5,0.333333,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
1118,0.333333,0.666667,1.0,0.000000,0.366741,0.508724,0.857143,0.0,0.5,0.000000,0.333333,0.000000,0.0,0.000000,0.000000,0.0,0.0


In [185]:
test_data

,Age,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Sex_female,Sex_male,Purpose_business,Purpose_car,Purpose_domestic appliances,Purpose_education,Purpose_furniture/equipment,Purpose_radio/TV,Purpose_repairs,Purpose_vacation/others
0,0.333333,0.666667,1.0,2.0,1.0,0.208948,0.375000,0.0,0.5,0.333333,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
1,0.333333,1.000000,1.0,0.5,1.0,0.202183,0.214286,0.0,0.5,0.000000,0.333333,0.0,0.0,0.000000,0.000000,0.0,0.0
2,0.666667,0.666667,1.0,0.5,0.5,0.095475,0.535714,0.0,0.5,0.000000,0.000000,0.0,0.0,0.000000,0.333333,0.0,0.0
3,0.333333,0.666667,1.0,0.5,0.5,0.360968,0.375000,0.0,0.5,0.333333,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
4,0.000000,0.333333,1.0,1.5,1.0,0.286425,0.616071,0.0,0.5,0.000000,0.000000,0.0,0.0,0.000000,0.333333,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1.000000,0.666667,0.0,0.5,0.5,0.873444,1.178571,0.0,0.5,0.000000,0.000000,0.0,0.0,0.333333,0.000000,0.0,0.0
196,0.666667,0.666667,1.0,2.0,1.0,0.274555,0.375000,0.0,0.5,0.333333,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
197,0.000000,0.666667,1.0,0.5,2.0,0.125854,0.214286,0.0,0.5,0.000000,0.000000,0.0,0.0,0.000000,0.333333,0.0,0.0
198,0.333333,0.666667,1.0,2.0,0.5,0.775289,0.696429,0.0,0.5,0.000000,0.000000,0.0,0.0,0.333333,0.000000,0.0,0.0


In [186]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(train_data,train_labels)

GaussianNB()

In [187]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5, p = 2,weights='distance')
knn.fit(train_data, train_labels)


KNeighborsClassifier(weights='distance')

In [188]:
pred1 = gnb.predict(test_data)
print(pred1,len(np.where(pred1==1)[0]))
from sklearn.metrics import classification_report
print(classification_report(y_pred=pred1, y_true = test_labels))

[0 0 0 1 0 0 1 0 1 0 0 1 0 1 1 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 1
 1 0 0 1 0 0 1 0 0 1 1 0 0 0 0 1 1 1 0 1 0 1 1 0 1 1 0 0 0 0 0 0 1 1 1 1 1
 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 1 1 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 1
 0 1 0 0 1 0 0 0 1 0 1 0 0 0 0 0 1 0 1 1 0 1 1 0 1 0 1 1 1 1 1 0 1 1 1 0 0
 0 0 0 0 0 0 1 1 1 1 1 1 1 0 1 0 1 0 0 0 0 0 1 0 0 1 0 1 0 0 1 1 0 0 1 1 0
 1 1 0 0 0 0 1 1 1 1 1 0 0 0 0] 79
              precision    recall  f1-score   support

           0       0.81      0.70      0.75       140
           1       0.47      0.62      0.53        60

    accuracy                           0.68       200
   macro avg       0.64      0.66      0.64       200
weighted avg       0.71      0.68      0.69       200



In [189]:
pred = knn.predict(test_data)
print(pred,len(np.where(pred1==1)[0]))
from sklearn.metrics import classification_report
print(classification_report(y_pred=pred, y_true = test_labels))

[0 1 1 0 0 1 1 1 0 1 0 0 0 0 1 0 1 0 1 1 0 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 0
 1 0 1 1 0 0 0 0 0 1 1 0 0 0 0 1 1 1 0 0 0 1 0 0 0 1 1 0 0 0 0 1 1 0 1 0 0
 1 1 0 0 1 0 1 0 1 0 0 1 1 0 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 1 1 0 1 0 0 1 0 0 1 0 0 1 1 0 0
 0 1 0 0 1 1 1 1 1 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0] 79
              precision    recall  f1-score   support

           0       0.79      0.70      0.74       140
           1       0.45      0.57      0.50        60

    accuracy                           0.66       200
   macro avg       0.62      0.63      0.62       200
weighted avg       0.69      0.66      0.67       200



In [190]:
print(pred)
print(pred1)

[0 1 1 0 0 1 1 1 0 1 0 0 0 0 1 0 1 0 1 1 0 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 0
 1 0 1 1 0 0 0 0 0 1 1 0 0 0 0 1 1 1 0 0 0 1 0 0 0 1 1 0 0 0 0 1 1 0 1 0 0
 1 1 0 0 1 0 1 0 1 0 0 1 1 0 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 1 1 0 1 0 0 1 0 0 1 0 0 1 1 0 0
 0 1 0 0 1 1 1 1 1 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0]
[0 0 0 1 0 0 1 0 1 0 0 1 0 1 1 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 1
 1 0 0 1 0 0 1 0 0 1 1 0 0 0 0 1 1 1 0 1 0 1 1 0 1 1 0 0 0 0 0 0 1 1 1 1 1
 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 1 1 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 1
 0 1 0 0 1 0 0 0 1 0 1 0 0 0 0 0 1 0 1 1 0 1 1 0 1 0 1 1 1 1 1 0 1 1 1 0 0
 0 0 0 0 0 0 1 1 1 1 1 1 1 0 1 0 1 0 0 0 0 0 1 0 0 1 0 1 0 0 1 1 0 0 1 1 0
 1 1 0 0 0 0 1 1 1 1 1 0 0 0 0]
